## Timings of **pykalman** vs **KalmanTV**

In [8]:
from kalmantv import KalmanTV
from kalmantv.tests.kalmantv_eigen import KalmanTV as KalmanTV_c
from pykalman import standard as pks
import time
import numpy as np

In [5]:
def rand_vec(n):
    """Generate a random vector."""
    return np.random.randn(n)

def rand_mat(n, p=None, pd=True):
    """Generate a random matrix, positive definite if `pd = True`."""
    if p is None:
        p = n
    V = np.zeros((n, p), order='F')
    V[:] = np.random.randn(n, p)
    if (p == n) & pd:
        V[:] = np.matmul(V, V.T)
    return V

In [9]:
def filter_time(n_state, n_meas):
    mu_state_past = rand_vec(n_state)
    var_state_past = rand_mat(n_state)
    mu_state = rand_vec(n_state)
    wgt_state = rand_mat(n_state, pd=False)
    var_state = rand_mat(n_state)
    x_meas = rand_vec(n_meas)
    mu_meas = rand_vec(n_meas)
    wgt_meas = rand_mat(n_meas, n_state, pd=False)
    var_meas = rand_mat(n_meas)
    loops = max(1, int(10000/n_state))
    start = time.time()
    for i in range(loops):
        mu_state_pred, var_state_pred = (
            pks._filter_predict(
                wgt_state, var_state,
                mu_state, mu_state_past,
                var_state_past
                )
        )
        _, mu_state_filt, var_state_filt = (
            pks._filter_correct(
                wgt_meas, var_meas,
                mu_meas, mu_state_pred,
                var_state_pred, x_meas
                )
            )
    end = time.time()
    pyk_time = (end - start)/loops
    ktv = KalmanTV_c(n_meas, n_state)
    mu_state_pred2 = np.empty(n_state)
    var_state_pred2 = np.empty((n_state, n_state), order='F')
    mu_state_filt2 = np.empty(n_state)
    var_state_filt2 = np.empty((n_state, n_state), order='F')
    start = time.time()
    for i in range(loops):
        ktv.filter(mu_state_pred2, var_state_pred2,
               mu_state_filt2, var_state_filt2,
               mu_state_past, var_state_past,
               mu_state, wgt_state, var_state,
               x_meas, mu_meas, wgt_meas, var_meas)
    end = time.time()
    ktv_time = (end - start)/loops
    ktv = KalmanTV(n_meas, n_state)
    mu_state_pred2 = np.empty(n_state)
    var_state_pred2 = np.empty((n_state, n_state), order='F')
    mu_state_filt2 = np.empty(n_state)
    var_state_filt2 = np.empty((n_state, n_state), order='F')
    start = time.time()
    for i in range(loops):
        ktv.filter(mu_state_pred2, var_state_pred2,
               mu_state_filt2, var_state_filt2,
               mu_state_past, var_state_past,
               mu_state, wgt_state, var_state,
               x_meas, mu_meas, wgt_meas, var_meas)
    end = time.time()
    ktv_time2 = (end - start)/loops
    return pyk_time, ktv_time, ktv_time2

In [10]:
n_meas = 1
n_state = 4
t1, t2, t3 = filter_time(n_state, n_meas)
print("py/cy:{}".format(t1/t3))
print("py/c:{}".format(t1/t2))

py/cy:2.05470803038422
py/c:5.902690208036198


In [11]:
n_meas = 3
n_state = 6
t1, t2, t3 = filter_time(n_state, n_meas)
print("py/cy:{}".format(t1/t3))
print("py/c:{}".format(t1/t2))

py/cy:2.9633450982589156
py/c:7.501674296204666


In [12]:
n_meas = 500
n_state = 1000
t1, t2, t3 = filter_time(n_state, n_meas)
print("py/cy:{}".format(t1/t3))
print("py/c:{}".format(t1/t2))

py/cy:3.1285084338710023
py/c:0.48692980645041745


## Pure **pykalman** Test

In [ ]:
from pykalman import standard as pks
import time
import numpy as np

def rand_vec(n):
    """Generate a random vector."""
    return np.random.randn(n)

def rand_mat(n, p=None, pd=True):
    """Generate a random matrix, positive definite if `pd = True`."""
    if p is None:
        p = n
    V = np.zeros((n, p), order='F')
    V[:] = np.random.randn(n, p)
    if (p == n) & pd:
        V[:] = np.matmul(V, V.T)
    return V

In [ ]:
# problem dimensions
n_meas = 10
n_state = 2000
n_reps = 50

# simulate inputs
start = time.time()
mu_state_past = rand_vec(n_state)
var_state_past = rand_mat(n_state)
mu_state = rand_vec(n_state)
wgt_state = rand_mat(n_state, pd=False)
var_state = rand_mat(n_state)
x_meas = rand_vec(n_meas)
mu_meas = rand_vec(n_meas)
wgt_meas = rand_mat(n_meas, n_state, pd=False)
var_meas = rand_mat(n_meas)
end = time.time()
end - start

In [ ]:
start = time.time()
for ii in range(n_reps):
    mu_state_pred, var_state_pred = (
        pks._filter_predict(
            wgt_state, var_state,
            mu_state, mu_state_past,
            var_state_past
        )
    )
    _, mu_state_filt, var_state_filt = (
        pks._filter_correct(
            wgt_meas, var_meas,
            mu_meas, mu_state_pred,
            var_state_pred, x_meas
        )
    )
end = time.time()
end - start

In [ ]:
np.show_config()